In [2]:
import nemo.collections.asr as nemo_asr

speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained("nvidia/speakerverification_en_titanet_large")

(…)peakerverification_en_titanet_large.nemo:   0%|          | 0.00/102M [00:00<?, ?B/s]

[NeMo W 2024-01-13 13:34:40 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2024-01-13 13:34:40 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

[NeMo I 2024-01-13 13:34:40 features:289] PADDING: 16
[NeMo I 2024-01-13 13:34:41 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--speakerverification_en_titanet_large/snapshots/0dc382f40121a5fbd34db10a2bb04d826c2be6a8/speakerverification_en_titanet_large.nemo.


In [5]:
import pandas as pd
import torch
import os
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
df = pd.read_csv("/content/ROBOVOX_SP_CUP_2024/data/single-channel/signle-channel-trials.trl", sep="\t", header=None)

In [7]:
df.head()

,0,1
0,spk_71,b80bb95d16
1,spk_71,e64da37b33
2,spk_71,c22d8f9d8a
3,spk_71,20e087d5bf
4,spk_71,c5396d4af7


In [8]:
enrollment_path = "/content/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment"

In [27]:
file_list = [f for f in Path(enrollment_path).iterdir() if f.is_file()]

In [28]:
speaker_to_file_map = dict()

In [32]:
for enrollment in file_list:
  speaker = enrollment.name.split('-')[0]
  array = speaker_to_file_map.get(speaker, -1)

  if(array==-1):
    speaker_to_file_map[speaker] = [str(enrollment)]
  else:
    speaker_to_file_map[speaker].append(str(enrollment))

In [35]:
## Using normal similarity metric

# with open('output.txt', 'w') as output:
#   for i in range(len(df)):
#     speaker, test_file = df.iloc[i][0], df.iloc[i][1]

#     test_embedding = speaker_model.get_embedding(f"/content/ROBOVOX_SP_CUP_2024/data/single-channel/test/{test_file}.wav")

#     enrollment_list = speaker_to_file_map.get(speaker)

#     e_embedding_1 = speaker_model.get_embedding(enrollment_list[0])
#     e_embedding_2 = speaker_model.get_embedding(enrollment_list[1])
#     e_embedding_3 = speaker_model.get_embedding(enrollment_list[2])

#     average_embedding = torch.mean(torch.stack([e_embedding_1, e_embedding_2, e_embedding_3], dim=0), dim=0)

#     similarity = cosine_similarity(test_embedding.cpu(), average_embedding.cpu())[0,0]

#     output.write(f"{speaker}\t{test_file}\t{similarity}\n")

In [ ]:
# After changing the range of the similarity to [0, 1]

with open('output.txt', 'w') as output:
  for i in range(len(df)):
    speaker, test_file = df.iloc[i][0], df.iloc[i][1]

    test_embedding = speaker_model.get_embedding(f"/content/ROBOVOX_SP_CUP_2024/data/single-channel/test/{test_file}.wav").squeeze()

    enrollment_list = speaker_to_file_map.get(speaker)

    e_embedding_1 = speaker_model.get_embedding(enrollment_list[0])
    e_embedding_2 = speaker_model.get_embedding(enrollment_list[1])
    e_embedding_3 = speaker_model.get_embedding(enrollment_list[2])

    average_embedding = torch.mean(torch.stack([e_embedding_1, e_embedding_2, e_embedding_3], dim=0), dim=0).squeeze()

    test_embedding_normalized = test_embedding / torch.linalg.norm(test_embedding.cpu())
    average_embedding_normalized = average_embedding / torch.linalg.norm(average_embedding.cpu())

    similarity = torch.dot(test_embedding_normalized, average_embedding_normalized) / ((torch.dot(test_embedding_normalized, test_embedding_normalized) * torch.dot(average_embedding_normalized, average_embedding_normalized)) ** 0.5)
    similarity = (similarity + 1) / 2

    # similarity = cosine_similarity(test_embedding.cpu(), average_embedding.cpu())[0,0]

    output.write(f"{speaker}\t{test_file}\t{similarity}\n")